In [1]:
from os import getcwd
from os.path import dirname
import json
import os
import math
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()

In [2]:
run_path = dirname(getcwd())
run_path

'/private/var/py/Seq2Seq'

In [3]:
import sys
sys.path.append(run_path)

In [4]:
from utils.iterator import ExtendTextIterator

In [5]:
source_path = '../dataset/lcsts/word/sources.sample.txt'
target_path = '../dataset/lcsts/word/summaries.sample.txt'
source_dict = '../dataset/lcsts/word/vocabs.json'
target_dict = '../dataset/lcsts/word/vocabs.json'
batch_size = 3
n_words_source = encoder_vocab_size = 30000
n_words_target = decoder_vocab_size = 30000
encoder_max_time_steps = 80
decoder_max_time_steps = 20
hidden_units = 400
attention_units = 250
keep_prob = 0.8
embedding_size = 300

In [6]:
print(os.getcwd())

/private/var/py/Seq2Seq/notebooks


In [7]:
train_set = ExtendTextIterator(source=source_path,
                                       target=target_path,
                                       source_dict=source_dict,
                                       target_dict=target_dict,
                                       batch_size=batch_size,
                                       n_words_source=n_words_source,
                                       n_words_target=n_words_target,
                                       sort_by_length=False,
                                       split_sign=' ',
                                       max_length=None,
                                       )

In [8]:
train_set.reset()
for batch in train_set.next():
    source_batch, target_batch, source_extend_batch, target_extend_batch, oovs_max_size = batch
    break

In [9]:
with open(source_path, encoding='utf-8') as f:
    for index in range(batch_size):
        print(index, f.readline().strip())

0 日前 ， 方舟子 发文 直指 林志颖啊 旗下 爱碧丽 推销 假 保健品 ， 引起 哗然 。 调查 发现 ， 爱碧丽 没有 自己 的 生产 加工厂 。 其 胶原蛋白 饮品 无 核心 研发 ， 全部 代工 生产 。 号称 有 “ 逆 生长 ” 功效 的 爱碧丽 “ 梦幻 奇迹 限量 组 ” 售价 高达 1080 元 ， 实际成本 仅为 每瓶 4 元 ！
1 韩方 应对 路径 可以 概括 为 ： 企业 道歉 担责 ； 政府 公正 不 护短 ； 民间 祈福 关怀 。 他们 深知 形象 的 重要 ， 竭力 呵护 企业 品牌 和 国家 形象 。 正如 有 评论 ， 韩国 “ 政府 + 企业 + 民众 ” 三位一体 式 呵护 韩国 国家 形象 的 “ 苦心经营 ” ， 的确 有 值得 我们 借鉴 之 处 。
2 63 岁 退休 教师 谢淑华啊 ， 拉着 人力 板车 ， 历时 1 年 ， 走 了 2 万 4 千里路 ， 带 着 年 过 九旬 的 妈妈 环游 中国 ， 完成 了 妈妈 “ 一辈子 在 锅台 边转 ， 也 想 出去 走走 ” 的 心愿 。 她 说 ： “ 妈妈 愿意 出去 走走 ， 我 就 愿意 拉着 ， 孝心 不能 等 ， 能 走 多 远 就 走 多 远 。


In [10]:
with open(target_path, encoding='utf-8') as f:
    for index in range(batch_size):
        print(index, f.readline().strip())

0 林志颖啊 公司 疑涉 虚假 营销案件 无 厂房 无 研发
1 从 韩亚 航空 事故 看其 应对 路径
2 女子 用 板车 拉 九旬 老母 环游 中国 1 年 走 2 万 4 千里


In [11]:
word2id = json.loads(open(source_dict).read())

In [12]:
id2word = {}
for key, value in word2id.items():
    id2word[value] = key

In [13]:
from utils.funcs import prepare_pair_batch

In [14]:
source, source_len, target, target_len = prepare_pair_batch(
                            source_batch, target_batch,
                            encoder_max_time_steps,
                            decoder_max_time_steps)
                        
# Get a batch from training parallel data
source_extend, _, target_extend, _ = prepare_pair_batch(
    source_extend_batch, target_extend_batch,
    encoder_max_time_steps,
    decoder_max_time_steps)

In [15]:
source

array([[  171,     3, 12606,  4045,  8484,     2,   971,     2,  7331,
         1885, 10703,     3,   599, 16866,     5,   144,   108,     3,
            2,    87,    79,     4,   484, 14891,     5,    78, 19803,
        12730,   541,  1055,  1338,     3,   492, 12334,   484,     5,
         4072,    16,     8,  6081,  5247,     9,  6873,     4,     2,
            8,  9937,  4885,  7341,  2884,     9,  1643,  1010, 26876,
           66,     3,     2,  3436, 25527,    63,    66,    41,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1],
       [14407,  1665,  5341,   102, 11572,    19,    20,    80,  2122,
        15952,    22,   170,  5020,    30,     2,    22,  1560,  9404,
        10063,     5,   119, 29899,  1734,     4,   291,     3,     2,
        14023,    80,   340,    12,   112,  1734,     5,  8734,    16,
         1965,     3,   522,     8,   170,   872,    80,   872,  1174,
            

In [16]:
for item in source:
    str = ''
    for word in item:
        str += id2word[word]
    print(str)

日前，方舟子发文直指UNK旗下UNK推销假保健品，引起哗然。调查发现，UNK没有自己的生产加工厂。其胶原蛋白饮品无核心研发，全部代工生产。号称有“逆生长”功效的UNK“梦幻奇迹限量组”售价高达1080元，UNK仅为每瓶4元！EOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOS
韩方应对路径可以概括为：企业道歉担责；政府公正不UNK；民间祈福关怀。他们深知形象的重要，UNK呵护企业品牌和国家形象。正如有评论，韩国“政府+企业+民众”UNK式呵护韩国国家形象的“UNK”，的确有值得我们借鉴之处。EOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOS
63岁退休教师UNK，拉着人力板车，历时1年，走了2万4UNK，带着年过UNK的妈妈UNK中国，完成了妈妈“一辈子在UNKUNK，也想出去走走”的心愿。她说：“妈妈愿意出去走走，我就愿意拉着，孝心不能等，能走多远就走多远。EOSEOSEOSEOSEOS


In [17]:
source_len

array([62, 65, 75])

In [18]:
target

array([[    2,    54,     2,  2350,     2,   541,  9438,   541,  1338,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1],
       [   44, 12257,   918,   610,     2,  1665,  5341,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1],
       [  487,   150, 26962,  2049,     2,     2,     2,    21,    42,
           17,   400,    60,   196,    63, 13789,     1,     1,     1,
            1,     1]], dtype=int32)

In [19]:
for item in target:
    str = ''
    for word in item:
        str += id2word[word]
    print(str)

UNK公司UNK虚假UNK无厂房无研发EOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOS
从韩亚航空事故UNK应对路径EOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOS
女子用板车拉UNKUNKUNK中国1年走2万4千里EOSEOSEOSEOSEOS


In [20]:
target_len

array([ 9,  7, 15])

In [21]:
source_extend

array([[  171,     3, 12606,  4045,  8484, 30000,   971, 30001,  7331,
         1885, 10703,     3,   599, 16866,     5,   144,   108,     3,
        30001,    87,    79,     4,   484, 14891,     5,    78, 19803,
        12730,   541,  1055,  1338,     3,   492, 12334,   484,     5,
         4072,    16,     8,  6081,  5247,     9,  6873,     4, 30001,
            8,  9937,  4885,  7341,  2884,     9,  1643,  1010, 26876,
           66,     3, 30002,  3436, 25527,    63,    66,    41,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1],
       [14407,  1665,  5341,   102, 11572,    19,    20,    80,  2122,
        15952,    22,   170,  5020,    30, 30000,    22,  1560,  9404,
        10063,     5,   119, 29899,  1734,     4,   291,     3, 30001,
        14023,    80,   340,    12,   112,  1734,     5,  8734,    16,
         1965,     3,   522,     8,   170,   872,    80,   872,  1174,
            

In [22]:
target_extend

array([[30000,    54,     2,  2350,     2,   541,  9438,   541,  1338,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1],
       [   44, 12257,   918,   610,     2,  1665,  5341,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1],
       [  487,   150, 26962,  2049, 30002,     2, 30003,    21,    42,
           17,   400,    60,   196,    63, 13789,     1,     1,     1,
            1,     1]], dtype=int32)

In [23]:
config = {'source_vocabulary': 'dataset/lcsts/word/vocabs.json', 'target_vocabulary': 'dataset/lcsts/word/vocabs.json', 'source_train_data': 'dataset/lcsts/word/sources.train.txt', 'target_train_data': 'dataset/lcsts/word/summaries.train.txt', 'source_valid_data': 'dataset/lcsts/word/sources.eval.txt', 'target_valid_data': 'dataset/lcsts/word/summaries.eval.txt', 'model_class': 'pointer_generator', 'cell_type': 'gru', 'attention_type': 'bahdanau', 'hidden_units': 400, 'attention_units': 250, 'encoder_depth': 3, 'decoder_depth': 3, 'embedding_size': 300, 'encoder_vocab_size': 30000, 'decoder_vocab_size': 30000, 'use_residual': False, 'use_dropout': True, 'use_bidirectional': False, 'dropout_rate': 0.3, 'split_sign': ' ', 'learning_rate': 0.0002, 'max_gradient_norm': 1.0, 'batch_size': 256, 'max_epochs': 100000, 'max_load_batches': 20, 'encoder_max_time_steps': 80, 'decoder_max_time_steps': 20, 'display_freq': 5, 'save_freq': 2000, 'valid_freq': 400, 'optimizer_type': 'adam', 'model_dir': 'checkpoints/lcsts_word_pointer_generator', 'model_name': 'lcsts.ckpt', 'use_fp16': False, 'shuffle_each_epoch': False, 'sort_by_length': False, 'gpu': '-1', 'allow_soft_placement': True, 'log_device_placement': False, 'debug': True, 'logger_name': 'train', 'logger_format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'}

In [24]:
import logging
logging.basicConfig(level=logging.DEBUG, format=config['logger_format'])
logger = logging.getLogger(config['logger_format'])

In [25]:
encoder_inputs = tf.constant(source, dtype=tf.int32, shape=[batch_size, encoder_max_time_steps],
                                             name='encoder_inputs')
encoder_inputs

<tf.Tensor: id=0, shape=(3, 80), dtype=int32, numpy=
array([[  171,     3, 12606,  4045,  8484,     2,   971,     2,  7331,
         1885, 10703,     3,   599, 16866,     5,   144,   108,     3,
            2,    87,    79,     4,   484, 14891,     5,    78, 19803,
        12730,   541,  1055,  1338,     3,   492, 12334,   484,     5,
         4072,    16,     8,  6081,  5247,     9,  6873,     4,     2,
            8,  9937,  4885,  7341,  2884,     9,  1643,  1010, 26876,
           66,     3,     2,  3436, 25527,    63,    66,    41,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1],
       [14407,  1665,  5341,   102, 11572,    19,    20,    80,  2122,
        15952,    22,   170,  5020,    30,     2,    22,  1560,  9404,
        10063,     5,   119, 29899,  1734,     4,   291,     3,     2,
        14023,    80,   340,    12,   112,  1734,     5,  8734,    16,
         1965,     3,   522,  

In [26]:
encoder_inputs_length = tf.constant(source_len, dtype=tf.int32, shape=[batch_size],
                                                    name='encoder_inputs_length')
encoder_inputs_length

<tf.Tensor: id=2, shape=(3,), dtype=int32, numpy=array([62, 65, 75], dtype=int32)>

In [27]:
encoder_inputs_extend = tf.constant(source_extend, dtype=tf.int32, shape=[batch_size, encoder_max_time_steps],
                                             name='encoder_inputs_extend')
encoder_inputs_extend

<tf.Tensor: id=4, shape=(3, 80), dtype=int32, numpy=
array([[  171,     3, 12606,  4045,  8484, 30000,   971, 30001,  7331,
         1885, 10703,     3,   599, 16866,     5,   144,   108,     3,
        30001,    87,    79,     4,   484, 14891,     5,    78, 19803,
        12730,   541,  1055,  1338,     3,   492, 12334,   484,     5,
         4072,    16,     8,  6081,  5247,     9,  6873,     4, 30001,
            8,  9937,  4885,  7341,  2884,     9,  1643,  1010, 26876,
           66,     3, 30002,  3436, 25527,    63,    66,    41,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1],
       [14407,  1665,  5341,   102, 11572,    19,    20,    80,  2122,
        15952,    22,   170,  5020,    30, 30000,    22,  1560,  9404,
        10063,     5,   119, 29899,  1734,     4,   291,     3, 30001,
        14023,    80,   340,    12,   112,  1734,     5,  8734,    16,
         1965,     3,   522,  

In [28]:
decoder_inputs = tf.constant(target, dtype=tf.int32, shape=[batch_size, decoder_max_time_steps],
                                                 name='decoder_inputs')
decoder_inputs

<tf.Tensor: id=6, shape=(3, 20), dtype=int32, numpy=
array([[    2,    54,     2,  2350,     2,   541,  9438,   541,  1338,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1],
       [   44, 12257,   918,   610,     2,  1665,  5341,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1],
       [  487,   150, 26962,  2049,     2,     2,     2,    21,    42,
           17,   400,    60,   196,    63, 13789,     1,     1,     1,
            1,     1]], dtype=int32)>

In [29]:
decoder_inputs_extend = tf.constant(target_extend, dtype=tf.int32,
                                                        shape=[batch_size, decoder_max_time_steps],
                                                        name='decoder_inputs_extend')
decoder_inputs_extend

<tf.Tensor: id=8, shape=(3, 20), dtype=int32, numpy=
array([[30000,    54,     2,  2350,     2,   541,  9438,   541,  1338,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1],
       [   44, 12257,   918,   610,     2,  1665,  5341,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1],
       [  487,   150, 26962,  2049, 30002,     2, 30003,    21,    42,
           17,   400,    60,   196,    63, 13789,     1,     1,     1,
            1,     1]], dtype=int32)>

In [30]:
decoder_inputs_length = tf.constant(target_len, dtype=tf.int32, shape=[batch_size],
                                                        name='decoder_inputs_length')
decoder_inputs_length

<tf.Tensor: id=10, shape=(3,), dtype=int32, numpy=array([ 9,  7, 15], dtype=int32)>

In [31]:
from utils.config import GO, EOS

In [32]:
decoder_start_token = tf.ones(shape=[batch_size, 1], dtype=tf.int32) * GO
decoder_start_token

<tf.Tensor: id=16, shape=(3, 1), dtype=int32, numpy=
array([[0],
       [0],
       [0]], dtype=int32)>

In [33]:
decoder_end_token = tf.ones(shape=[batch_size, 1], dtype=tf.int32) * EOS
decoder_end_token

<tf.Tensor: id=20, shape=(3, 1), dtype=int32, numpy=
array([[1],
       [1],
       [1]], dtype=int32)>

In [34]:
decoder_inputs_train_length = decoder_inputs_length + 1
decoder_inputs_train_length

<tf.Tensor: id=22, shape=(3,), dtype=int32, numpy=array([10,  8, 16], dtype=int32)>

In [35]:
decoder_inputs_train = tf.concat([decoder_start_token, decoder_inputs], axis=-1)
decoder_inputs_train

<tf.Tensor: id=25, shape=(3, 21), dtype=int32, numpy=
array([[    0,     2,    54,     2,  2350,     2,   541,  9438,   541,
         1338,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1],
       [    0,    44, 12257,   918,   610,     2,  1665,  5341,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1],
       [    0,   487,   150, 26962,  2049,     2,     2,     2,    21,
           42,    17,   400,    60,   196,    63, 13789,     1,     1,
            1,     1,     1]], dtype=int32)>

In [36]:
decoder_targets_train = tf.concat([decoder_inputs_extend, decoder_end_token], axis=-1)
decoder_targets_train

<tf.Tensor: id=27, shape=(3, 21), dtype=int32, numpy=
array([[30000,    54,     2,  2350,     2,   541,  9438,   541,  1338,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1],
       [   44, 12257,   918,   610,     2,  1665,  5341,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1],
       [  487,   150, 26962,  2049, 30002,     2, 30003,    21,    42,
           17,   400,    60,   196,    63, 13789,     1,     1,     1,
            1,     1,     1]], dtype=int32)>

In [37]:
decoder_targets_train_length = decoder_inputs_length + 1
decoder_targets_train_length

<tf.Tensor: id=29, shape=(3,), dtype=int32, numpy=array([10,  8, 16], dtype=int32)>

In [38]:
attention_u = tf.get_variable(name='attention_u', shape=[hidden_units, attention_units],
                                               initializer=tf.truncated_normal_initializer)
attention_u

<tf.Variable 'attention_u:0' shape=(400, 250) dtype=float32, numpy=
array([[-1.0492138 , -0.94521177,  1.7928405 , ..., -0.880785  ,
        -0.20985304, -0.19303153],
       [ 0.16966423,  0.24342975, -0.59975624, ...,  0.6938214 ,
         0.6392404 ,  0.3064716 ],
       [ 1.0081607 ,  0.5103068 ,  1.6420931 , ...,  0.5490913 ,
        -1.3614547 , -1.1382138 ],
       ...,
       [-0.29396364, -0.32022947,  1.1126    , ...,  0.3210742 ,
        -0.15002817, -0.3049761 ],
       [ 1.9523152 ,  1.0639052 , -1.5761958 , ..., -0.25431964,
        -0.5822448 , -0.02893735],
       [ 0.22364895, -0.9295583 , -0.8069407 , ..., -0.3192556 ,
        -0.07175918,  1.3477153 ]], dtype=float32)>

In [39]:
attention_w = tf.get_variable(name='attention_w', shape=[hidden_units, attention_units],
                                               initializer=tf.truncated_normal_initializer)
attention_w

<tf.Variable 'attention_w:0' shape=(400, 250) dtype=float32, numpy=
array([[-1.051774  ,  0.5712928 ,  0.01953494, ..., -0.05668771,
        -1.2585914 ,  0.3528204 ],
       [ 1.9592196 ,  0.505376  ,  1.2478019 , ..., -0.05724112,
         0.04452757, -1.0796586 ],
       [ 0.2333244 ,  0.6517117 ,  0.24797884, ...,  0.4262939 ,
        -1.9031957 , -1.457282  ],
       ...,
       [ 1.1138618 ,  0.70870805, -1.2515165 , ...,  0.19356562,
        -0.32351792, -0.00391081],
       [ 0.9983331 ,  0.35130507, -1.9739978 , ..., -0.2603574 ,
        -0.27288288, -0.793293  ],
       [ 0.24361834, -0.54674226,  0.28884152, ..., -0.9445787 ,
         0.28886974, -0.532169  ]], dtype=float32)>

In [40]:
attention_v = tf.get_variable(name='attention_v', shape=[attention_units, 1],
                                               initializer=tf.truncated_normal_initializer)
attention_v

<tf.Variable 'attention_v:0' shape=(250, 1) dtype=float32, numpy=
array([[ 0.05603277],
       [-1.712727  ],
       [ 0.57727104],
       [-0.08539746],
       [ 0.19679855],
       [-0.2734202 ],
       [-0.15233557],
       [-0.55905026],
       [-0.11236904],
       [ 0.1584516 ],
       [-0.23677616],
       [ 1.492124  ],
       [ 0.38693535],
       [-1.1037121 ],
       [ 1.6458981 ],
       [ 0.46952388],
       [-0.8889546 ],
       [-0.39114863],
       [-1.0151786 ],
       [ 1.5056871 ],
       [-1.5372983 ],
       [-1.0029012 ],
       [ 1.2778019 ],
       [ 0.8149569 ],
       [-1.4246523 ],
       [-1.0350918 ],
       [-0.8102646 ],
       [ 0.794233  ],
       [-0.825473  ],
       [ 1.1824037 ],
       [ 1.1174212 ],
       [ 1.3526866 ],
       [ 0.40930828],
       [ 0.00382559],
       [-1.0927476 ],
       [ 0.70863825],
       [-0.10200632],
       [-0.08465575],
       [ 1.2001033 ],
       [ 0.2736706 ],
       [ 1.1471213 ],
       [-0.1057853 ],
       [-1

In [41]:
def build_single_cell():
    """
    build single cell, lstm or gru or RNN
    :return: GRUCell or LSTMCell or RNNCell
    """
    cell = tf.nn.rnn_cell.GRUCell(hidden_units, name='single_cell')
    cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, dtype=tf.float32, output_keep_prob=keep_prob)
    return cell

In [42]:
encoder_embeddings = tf.get_variable(name='encoder_embeddings',
                                                      shape=[encoder_vocab_size, embedding_size],
                                                      dtype=tf.float32,
                                                      initializer=tf.random_uniform_initializer(-math.sqrt(3),
                                                                                                math.sqrt(3),
                                                                                                dtype=tf.float32))
encoder_embeddings

<tf.Variable 'encoder_embeddings:0' shape=(30000, 300) dtype=float32, numpy=
array([[-0.33135104,  1.2932965 ,  0.49160588, ...,  0.1237781 ,
         1.2399164 ,  0.9902569 ],
       [ 0.03958035,  0.6401025 , -0.25655162, ..., -1.1946968 ,
        -0.07999814, -1.2892268 ],
       [ 0.59704673,  1.7064325 , -1.2312349 , ..., -0.5596683 ,
        -1.4941274 , -0.53203964],
       ...,
       [-0.14740479,  0.22732651, -1.1053466 , ..., -1.2719038 ,
         0.51511896, -1.6430098 ],
       [-1.6445048 , -0.71983385, -0.9669049 , ...,  0.16478229,
        -0.6439855 , -1.6020222 ],
       [ 0.55665743, -0.64968336,  0.01334083, ..., -1.4715934 ,
        -1.3455594 , -0.9551258 ]], dtype=float32)>

In [43]:
encoder_inputs_embedded = tf.nn.embedding_lookup(params=encoder_embeddings,
                                                                  ids=encoder_inputs,
                                                                  name='inputs_embedded')
encoder_inputs_embedded

<tf.Tensor: id=90, shape=(3, 80, 300), dtype=float32, numpy=
array([[[ 1.0259341 , -1.5344692 ,  0.0889014 , ...,  1.412222  ,
         -0.7966666 , -0.13741505],
        [ 0.31052244,  0.79650104, -1.3038733 , ...,  0.5769154 ,
         -0.38754904,  0.5477456 ],
        [ 0.44872797, -1.704892  , -0.40272343, ...,  1.189387  ,
         -1.5284381 ,  1.0062379 ],
        ...,
        [ 0.03958035,  0.6401025 , -0.25655162, ..., -1.1946968 ,
         -0.07999814, -1.2892268 ],
        [ 0.03958035,  0.6401025 , -0.25655162, ..., -1.1946968 ,
         -0.07999814, -1.2892268 ],
        [ 0.03958035,  0.6401025 , -0.25655162, ..., -1.1946968 ,
         -0.07999814, -1.2892268 ]],

       [[ 0.8250679 , -0.4273305 , -0.32245982, ...,  1.4062928 ,
         -1.7073971 ,  0.20285583],
        [ 0.27085912, -0.9915251 , -0.87660027, ...,  0.9491612 ,
         -0.47214162, -0.5419208 ],
        [ 1.7027401 ,  0.9205295 ,  0.28765023, ...,  0.37938416,
          1.6183618 ,  0.73046434],
      

In [44]:
encoder_inputs_embedded_dense = tf.layers.dense(inputs=encoder_inputs_embedded,
                                                                 units=hidden_units,
                                                                 use_bias=False,
                                                                 name='inputs_embedded_dense')
encoder_inputs_embedded_dense

<tf.Tensor: id=116, shape=(3, 80, 400), dtype=float32, numpy=
array([[[-0.96929127, -1.0713835 , -0.6496213 , ..., -1.7611964 ,
          0.19962682,  1.3500576 ],
        [-1.0060122 , -1.1599994 , -0.9995326 , ..., -0.19844165,
         -0.0937869 ,  0.7775981 ],
        [-0.6937088 ,  0.14513214,  0.08335346, ..., -0.06558988,
         -0.9692128 , -0.36466256],
        ...,
        [ 0.7676285 , -0.92727166,  0.48458207, ...,  1.3237509 ,
          0.0674155 , -0.06102741],
        [ 0.7676285 , -0.92727166,  0.48458207, ...,  1.3237509 ,
          0.0674155 , -0.06102741],
        [ 0.7676285 , -0.92727166,  0.48458207, ...,  1.3237509 ,
          0.0674155 , -0.06102741]],

       [[ 0.27583587,  0.559545  ,  1.5521797 , ..., -0.36191338,
         -1.5807288 , -1.0333691 ],
        [ 0.17923394,  0.8411    ,  0.09171981, ..., -0.262371  ,
         -0.03386144,  0.28198877],
        [-2.2956567 , -1.0568991 ,  0.37663758, ..., -0.21783376,
          0.5242923 ,  0.7425685 ],
     

In [45]:
encoder_outputs, encoder_last_state = tf.nn.dynamic_rnn(cell=build_single_cell(), inputs=encoder_inputs_embedded_dense,
                                                                        sequence_length=encoder_inputs_length,
                                                                        dtype=tf.float32)
encoder_outputs

<tf.Tensor: id=3883, shape=(3, 80, 400), dtype=float32, numpy=
array([[[ 0.06181268, -0.        ,  0.40571627, ...,  0.01990418,
          0.02856621,  0.06978092],
        [ 0.05748754,  0.08823056,  0.36928   , ...,  0.        ,
         -0.        ,  0.        ],
        [ 0.02060231, -0.13810173,  0.        , ...,  0.20801339,
         -0.        ,  0.04893079],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.19726841,  0.24974464, -0.        , ..., -0.14554754,
          0.        ,  0.06457989],
        [-0.        ,  0.23995055, -0.        , ...,  0.26970536,
          0.4367578 ,  0.03818594],
        [ 0.49916354,  0.22829302, -0.        , ...,  0.1283324 ,
          0.29486018, -0.00535558],
    

In [46]:
encoder_last_state

<tf.Tensor: id=3871, shape=(3, 400), dtype=float32, numpy=
array([[ 0.36122486,  0.4930659 , -0.38498944, ...,  0.10383439,
        -0.00102501,  0.2837364 ],
       [-0.32999456,  0.3408171 , -0.31316215, ..., -0.34984162,
        -0.1631628 , -0.0618646 ],
       [ 0.20245185, -0.19808918,  0.1531785 , ..., -0.38401106,
        -0.09420335, -0.05021207]], dtype=float32)>

In [47]:
decoder_initial_state = encoder_last_state

In [48]:
decoder_cell = build_single_cell()

In [49]:
decoder_embeddings = tf.get_variable(name='decoder_embeddings',
                                                      shape=[decoder_vocab_size, embedding_size],
                                                      dtype=tf.float32,
                                                      initializer=tf.random_uniform_initializer(-math.sqrt(3),
                                                                                                math.sqrt(3),
                                                                                                dtype=tf.float32))

In [50]:
decoder_embeddings

<tf.Variable 'decoder_embeddings:0' shape=(30000, 300) dtype=float32, numpy=
array([[ 1.1032504 ,  0.6729306 ,  0.8648964 , ..., -0.46854818,
         1.336697  , -0.5914562 ],
       [-1.0323112 ,  0.9373454 ,  0.23000085, ..., -0.54759896,
         0.10571849,  0.0998863 ],
       [ 0.5316478 , -1.5282779 , -0.36714458, ...,  0.6276916 ,
        -1.6423478 , -0.6969657 ],
       ...,
       [-0.05392873,  0.6871258 , -0.44261384, ..., -1.6662012 ,
        -0.27153933,  1.2755944 ],
       [-0.7410282 ,  0.04979181, -1.062416  , ..., -0.46514785,
         0.56527746,  0.63719   ],
       [-0.9248076 ,  0.72404397, -0.9130876 , ..., -1.6224151 ,
         0.8380698 ,  0.49150622]], dtype=float32)>

In [51]:
encoder_outputs_unstack = tf.unstack(encoder_outputs, axis=1)
encoder_outputs_unstack

[<tf.Tensor: id=3903, shape=(3, 400), dtype=float32, numpy=
 array([[ 0.06181268, -0.        ,  0.40571627, ...,  0.01990418,
          0.02856621,  0.06978092],
        [-0.19726841,  0.24974464, -0.        , ..., -0.14554754,
          0.        ,  0.06457989],
        [ 0.2167993 ,  0.09907298,  0.40990922, ..., -0.08291218,
          0.        ,  0.17482899]], dtype=float32)>,
 <tf.Tensor: id=3904, shape=(3, 400), dtype=float32, numpy=
 array([[ 0.05748754,  0.08823056,  0.36928   , ...,  0.        ,
         -0.        ,  0.        ],
        [-0.        ,  0.23995055, -0.        , ...,  0.26970536,
          0.4367578 ,  0.03818594],
        [ 0.        ,  0.05688686,  0.42660564, ..., -0.05173567,
          0.24725585,  0.01976985]], dtype=float32)>,
 <tf.Tensor: id=3905, shape=(3, 400), dtype=float32, numpy=
 array([[ 0.02060231, -0.13810173,  0.        , ...,  0.20801339,
         -0.        ,  0.04893079],
        [ 0.49916354,  0.22829302, -0.        , ...,  0.1283324 ,
    

In [52]:
decoder_inputs_embedded = tf.nn.embedding_lookup(params=decoder_embeddings,
                                                                      ids=decoder_inputs_train)
decoder_inputs_embedded

<tf.Tensor: id=4064, shape=(3, 21, 300), dtype=float32, numpy=
array([[[ 1.1032504 ,  0.6729306 ,  0.8648964 , ..., -0.46854818,
          1.336697  , -0.5914562 ],
        [ 0.5316478 , -1.5282779 , -0.36714458, ...,  0.6276916 ,
         -1.6423478 , -0.6969657 ],
        [ 1.4907585 , -1.0475187 , -0.8871174 , ...,  0.8128804 ,
          1.4053346 ,  0.4885975 ],
        ...,
        [-1.0323112 ,  0.9373454 ,  0.23000085, ..., -0.54759896,
          0.10571849,  0.0998863 ],
        [-1.0323112 ,  0.9373454 ,  0.23000085, ..., -0.54759896,
          0.10571849,  0.0998863 ],
        [-1.0323112 ,  0.9373454 ,  0.23000085, ..., -0.54759896,
          0.10571849,  0.0998863 ]],

       [[ 1.1032504 ,  0.6729306 ,  0.8648964 , ..., -0.46854818,
          1.336697  , -0.5914562 ],
        [ 1.5765346 ,  0.2981025 , -0.7901081 , ...,  0.35759127,
         -0.03482676,  1.1342467 ],
        [ 1.1861888 , -0.10126638, -1.2773354 , ...,  0.36510432,
          1.6605273 ,  0.01598704],
    

In [53]:
decoder_inputs_embedded_unstack = tf.unstack(decoder_inputs_embedded, axis=1)
decoder_inputs_embedded_unstack

[<tf.Tensor: id=4066, shape=(3, 300), dtype=float32, numpy=
 array([[ 1.1032504 ,  0.6729306 ,  0.8648964 , -0.11804223,  1.2567173 ,
          1.1463243 ,  0.27658188, -0.07266688, -0.58676994,  0.06610513,
          0.07313073,  0.2160393 , -0.6356467 , -0.45327508, -1.4310782 ,
          1.4768184 ,  1.5998863 , -1.0045369 , -1.3867881 ,  0.16658235,
         -0.70251906,  0.16755319,  1.2209512 , -0.7145897 , -0.06601262,
          0.5350472 , -1.4412963 ,  0.8765491 , -0.11197424, -1.5714455 ,
          0.12858284, -1.3769594 , -0.1317221 , -1.0246189 , -0.02220941,
          0.80291617, -0.9275046 ,  1.6435503 ,  1.3317672 ,  0.98146594,
          0.30602753, -0.8966207 , -0.6932248 ,  0.1863954 ,  0.4943415 ,
          1.3082291 , -0.02822125, -1.2225902 ,  1.2513593 , -0.62662244,
          0.84195554,  1.2569948 , -0.97829163,  0.10803223,  0.1894604 ,
          0.51117885, -1.0105684 , -0.23608398, -0.692379  , -0.8505888 ,
         -0.9206624 , -0.16828215, -0.5357059 ,  0.3

In [54]:
def attention(prev_state, encoder_outputs):
    """
    calculate attention result
    :param prev_state: prev state
    :param encoder_outputs: encoder outputs
    :return: attention result
    """
    e_i = []
    c_i = []
    # encoder_outputs: encoder_time_steps * [batch_size, hidden_units]
    # output: [batch_size, hidden_units]
    for output in encoder_outputs:
        # e_i_j: [batch_size, 1]
        e_i_j = tf.matmul(
            # tanh: [batch_size, attention_units]
            tf.tanh(
                # prev_state: [batch_size, hidden_units]
                # attention_w: [hidden_units, attention_units]
                tf.matmul(prev_state, attention_w) +
                # output: [batch_size, hidden_units]
                # attention_u: [hidden_units, attention_units]
                tf.matmul(output, attention_u)
            ),
            # attention_v: [attention_units, 1]
            attention_v)
        # e_i: encoder_time_steps * [batch_size, 1]
        e_i.append(e_i_j)
    # e_i: [batch_size, encoder_time_steps]
    e_i = tf.concat(e_i, axis=1)
    # alpha_i: [batch_size, encoder_time_steps]
    alpha_i = tf.nn.softmax(e_i, axis=-1)
    # alpha_i_split: encoder_time_steps * [batch_size, 1]
    alpha_i_split = tf.split(alpha_i, alpha_i.shape[-1], axis=-1)
    # alpha_i_split: encoder_time_steps * [batch_size, 1]
    # encoder_outputs: encoder_time_steps * [batch_size, hidden_units]
    for alpha_i_j, output in zip(alpha_i_split, encoder_outputs):
        # alpha_i_j: [batch_size, 1]
        # output: [batch_size, hidden_units]
        # c_i_j: [batch_size, hidden_units]
        c_i_j = tf.multiply(alpha_i_j, output)
        # c_i: encoder_time_steps * [batch_size, hidden_units]
        c_i.append(c_i_j)
    # c_i: [batch_size, hidden_units]
    c_i = tf.reduce_sum(c_i, axis=0)
    # c_i: [batch_size, hidden_units]
    # alpha_i: [batch_size, encoder_time_steps]
    return c_i, alpha_i

In [55]:
state = decoder_initial_state
c_i_group = []
alpha_i_group = []
outputs_group = []
p_gen_group = []
state_group = []
with tf.variable_scope('loop', reuse=tf.AUTO_REUSE):
    for i, inputs in enumerate(decoder_inputs_embedded_unstack):
        c_i, alpha_i = attention(state, encoder_outputs=encoder_outputs_unstack)
        c_i_group.append(c_i)
        alpha_i_group.append(alpha_i)
        p_gen_dense = tf.layers.dense(tf.concat([c_i, state, inputs], axis=-1),
                                                      units=1,
                                                      name='p_gen_dense')
        p_gen = tf.nn.sigmoid(p_gen_dense, name='p_gen_sigmoid')
        p_gen_group.append(p_gen)
        inputs = tf.concat([inputs, c_i], axis=1)
        outputs, state = decoder_cell(inputs=inputs, state=state)
        outputs_group.append(outputs)
        state_group.append(state)
        # p_gen_dense: [batch_size, 1]

In [56]:
outputs_group

[<tf.Tensor: id=5011, shape=(3, 400), dtype=float32, numpy=
 array([[ 0.27089053,  0.6993327 , -0.34142232, ...,  0.        ,
          0.00398639,  0.39074114],
        [-0.20870031,  0.5522032 , -0.2007737 , ...,  0.16055465,
         -0.04971617, -0.        ],
        [ 0.27349955, -0.02468547,  0.        , ..., -0.1291104 ,
         -0.01418122,  0.01951301]], dtype=float32)>,
 <tf.Tensor: id=5870, shape=(3, 400), dtype=float32, numpy=
 array([[ 0.05144771,  0.7281443 , -0.37356114, ...,  0.05843233,
         -0.04952332,  0.47577962],
        [ 0.01655357,  0.        , -0.08747415, ..., -0.        ,
          0.21920867,  0.03197804],
        [ 0.        , -0.48752922,  0.23391642, ..., -0.08230341,
          0.        ,  0.        ]], dtype=float32)>,
 <tf.Tensor: id=6729, shape=(3, 400), dtype=float32, numpy=
 array([[ 0.        ,  0.750198  , -0.06202815, ..., -0.20453878,
          0.15689671,  0.45554897],
        [-0.09942224,  0.51839924, -0.09165209, ...,  0.38226095,
    

In [57]:
p_gen_group

[<tf.Tensor: id=4939, shape=(3, 1), dtype=float32, numpy=
 array([[0.87957317],
        [0.7754687 ],
        [0.8110325 ]], dtype=float32)>,
 <tf.Tensor: id=5840, shape=(3, 1), dtype=float32, numpy=
 array([[0.33255798],
        [0.6392306 ],
        [0.23890685]], dtype=float32)>,
 <tf.Tensor: id=6699, shape=(3, 1), dtype=float32, numpy=
 array([[0.2316914],
        [0.486628 ],
        [0.6512805]], dtype=float32)>,
 <tf.Tensor: id=7558, shape=(3, 1), dtype=float32, numpy=
 array([[0.5674238 ],
        [0.38187414],
        [0.6187975 ]], dtype=float32)>,
 <tf.Tensor: id=8417, shape=(3, 1), dtype=float32, numpy=
 array([[0.47214603],
        [0.4381661 ],
        [0.52042013]], dtype=float32)>,
 <tf.Tensor: id=9276, shape=(3, 1), dtype=float32, numpy=
 array([[0.45183408],
        [0.33085093],
        [0.3852602 ]], dtype=float32)>,
 <tf.Tensor: id=10135, shape=(3, 1), dtype=float32, numpy=
 array([[0.26367372],
        [0.5515943 ],
        [0.41685528]], dtype=float32)>,
 <tf.Ten

In [58]:
len(p_gen_group)

21

In [59]:
outputs_logits_group = [tf.layers.dense(inputs=outputs,
                                                         units=decoder_vocab_size,
                                                         name='outputs_dense') for outputs in outputs_group]

In [60]:
len(outputs_logits_group)

21

In [61]:
outputs_logits_group[-1]

<tf.Tensor: id=22718, shape=(3, 30000), dtype=float32, numpy=
array([[-0.08420172, -0.09884591,  0.00238671, ..., -0.00528505,
         0.0608087 , -0.04567662],
       [-0.09521344, -0.02381391, -0.01921143, ...,  0.07344827,
         0.15473503, -0.02141917],
       [-0.00781592, -0.04458879,  0.02142014, ...,  0.0294504 ,
         0.07853767, -0.02242444]], dtype=float32)>

In [62]:
vocab_distribution_group = [tf.nn.softmax(outputs_logits, axis=-1) for outputs_logits in outputs_logits_group]

In [63]:
vocab_distribution_argmax_group = [tf.argmax(vocab_distribution, -1) for vocab_distribution in vocab_distribution_group]

In [64]:
vocab_distribution_argmax_group

[<tf.Tensor: id=22741, shape=(3,), dtype=int64, numpy=array([27414, 27510, 13788])>,
 <tf.Tensor: id=22742, shape=(3,), dtype=int64, numpy=array([15721, 16716,  4924])>,
 <tf.Tensor: id=22743, shape=(3,), dtype=int64, numpy=array([ 3853,  8111, 19744])>,
 <tf.Tensor: id=22744, shape=(3,), dtype=int64, numpy=array([22121, 11399, 19424])>,
 <tf.Tensor: id=22745, shape=(3,), dtype=int64, numpy=array([ 8816,  9906, 26916])>,
 <tf.Tensor: id=22746, shape=(3,), dtype=int64, numpy=array([ 1985,  9482, 28795])>,
 <tf.Tensor: id=22747, shape=(3,), dtype=int64, numpy=array([18933, 25711, 14216])>,
 <tf.Tensor: id=22748, shape=(3,), dtype=int64, numpy=array([ 5199, 17060,  5147])>,
 <tf.Tensor: id=22749, shape=(3,), dtype=int64, numpy=array([25159,  3884, 26790])>,
 <tf.Tensor: id=22750, shape=(3,), dtype=int64, numpy=array([11754, 27987,  2838])>,
 <tf.Tensor: id=22751, shape=(3,), dtype=int64, numpy=array([13870, 29297,  9773])>,
 <tf.Tensor: id=22752, shape=(3,), dtype=int64, numpy=array([2281

In [65]:
attention_distribution_group = alpha_i_group
attention_distribution_group[-1]

<tf.Tensor: id=21974, shape=(3, 80), dtype=float32, numpy=
array([[1.76058919e-08, 4.11916412e-09, 9.16569880e-13, 4.91414687e-09,
        3.74457863e-06, 5.02056810e-06, 1.13563074e-04, 5.32638694e-07,
        2.14910626e-01, 6.84660108e-06, 1.60682719e-12, 1.00639897e-06,
        4.46636044e-08, 1.10554530e-13, 5.58932969e-11, 3.28192384e-09,
        2.43395548e-09, 1.33448130e-09, 1.98727298e-06, 8.98698054e-05,
        1.21476141e-05, 7.24559213e-09, 1.25806073e-05, 2.81278423e-09,
        7.61677121e-09, 9.46984164e-06, 5.29884459e-13, 1.34436891e-06,
        1.00493326e-03, 6.04727745e-07, 1.76109040e-07, 7.69760788e-09,
        4.29595531e-07, 5.97114047e-14, 8.35728795e-12, 1.15757088e-04,
        5.89445940e-12, 1.46924237e-10, 3.31048210e-11, 1.20419733e-12,
        2.29464614e-13, 2.10576578e-13, 2.01395541e-13, 5.46284795e-09,
        3.39846595e-10, 5.46503687e-09, 1.86696093e-06, 9.80353434e-06,
        7.79516876e-01, 1.11351426e-07, 4.55398030e-09, 9.06647927e-07,
     

In [66]:
attention_distribution_argmax_group = [tf.argmax(attention_distribution, -1) for attention_distribution in attention_distribution_group]

In [67]:
attention_distribution_sum_group = [tf.reduce_sum(attention_distribution, -1) for attention_distribution in attention_distribution_group]

In [68]:
attention_distribution_sum_group

[<tf.Tensor: id=22805, shape=(3,), dtype=float32, numpy=array([1.        , 1.        , 0.99999994], dtype=float32)>,
 <tf.Tensor: id=22806, shape=(3,), dtype=float32, numpy=array([1.        , 0.99999994, 1.        ], dtype=float32)>,
 <tf.Tensor: id=22807, shape=(3,), dtype=float32, numpy=array([1.0000001, 1.0000001, 1.       ], dtype=float32)>,
 <tf.Tensor: id=22808, shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Tensor: id=22809, shape=(3,), dtype=float32, numpy=array([0.99999994, 1.0000001 , 1.0000001 ], dtype=float32)>,
 <tf.Tensor: id=22810, shape=(3,), dtype=float32, numpy=array([1.0000001 , 0.99999994, 1.        ], dtype=float32)>,
 <tf.Tensor: id=22811, shape=(3,), dtype=float32, numpy=array([1.0000001, 0.9999999, 0.9999999], dtype=float32)>,
 <tf.Tensor: id=22812, shape=(3,), dtype=float32, numpy=array([1.0000001, 0.9999998, 1.0000001], dtype=float32)>,
 <tf.Tensor: id=22813, shape=(3,), dtype=float32, numpy=array([1.        , 0.99999994, 1.        

In [69]:
attention_distribution_argmax_group

[<tf.Tensor: id=22784, shape=(3,), dtype=int64, numpy=array([48, 54, 41])>,
 <tf.Tensor: id=22785, shape=(3,), dtype=int64, numpy=array([48, 31, 31])>,
 <tf.Tensor: id=22786, shape=(3,), dtype=int64, numpy=array([31, 31, 41])>,
 <tf.Tensor: id=22787, shape=(3,), dtype=int64, numpy=array([31, 45, 13])>,
 <tf.Tensor: id=22788, shape=(3,), dtype=int64, numpy=array([47, 53, 50])>,
 <tf.Tensor: id=22789, shape=(3,), dtype=int64, numpy=array([47, 53, 36])>,
 <tf.Tensor: id=22790, shape=(3,), dtype=int64, numpy=array([47, 15, 40])>,
 <tf.Tensor: id=22791, shape=(3,), dtype=int64, numpy=array([58, 44, 35])>,
 <tf.Tensor: id=22792, shape=(3,), dtype=int64, numpy=array([47, 15, 21])>,
 <tf.Tensor: id=22793, shape=(3,), dtype=int64, numpy=array([47,  1, 21])>,
 <tf.Tensor: id=22794, shape=(3,), dtype=int64, numpy=array([28,  1, 35])>,
 <tf.Tensor: id=22795, shape=(3,), dtype=int64, numpy=array([18,  1, 40])>,
 <tf.Tensor: id=22796, shape=(3,), dtype=int64, numpy=array([18, 10, 36])>,
 <tf.Tensor:

In [70]:
p_gen_group

[<tf.Tensor: id=4939, shape=(3, 1), dtype=float32, numpy=
 array([[0.87957317],
        [0.7754687 ],
        [0.8110325 ]], dtype=float32)>,
 <tf.Tensor: id=5840, shape=(3, 1), dtype=float32, numpy=
 array([[0.33255798],
        [0.6392306 ],
        [0.23890685]], dtype=float32)>,
 <tf.Tensor: id=6699, shape=(3, 1), dtype=float32, numpy=
 array([[0.2316914],
        [0.486628 ],
        [0.6512805]], dtype=float32)>,
 <tf.Tensor: id=7558, shape=(3, 1), dtype=float32, numpy=
 array([[0.5674238 ],
        [0.38187414],
        [0.6187975 ]], dtype=float32)>,
 <tf.Tensor: id=8417, shape=(3, 1), dtype=float32, numpy=
 array([[0.47214603],
        [0.4381661 ],
        [0.52042013]], dtype=float32)>,
 <tf.Tensor: id=9276, shape=(3, 1), dtype=float32, numpy=
 array([[0.45183408],
        [0.33085093],
        [0.3852602 ]], dtype=float32)>,
 <tf.Tensor: id=10135, shape=(3, 1), dtype=float32, numpy=
 array([[0.26367372],
        [0.5515943 ],
        [0.41685528]], dtype=float32)>,
 <tf.Ten

In [71]:
attention_distribution_group

[<tf.Tensor: id=4749, shape=(3, 80), dtype=float32, numpy=
 array([[3.9842826e-06, 2.4376622e-08, 1.5173105e-08, 9.3455469e-12,
         5.5006103e-16, 5.2598721e-09, 1.5100781e-08, 5.9595079e-10,
         2.8024598e-11, 1.4715696e-13, 5.8243117e-17, 1.0552802e-09,
         3.6235241e-09, 9.2218236e-13, 4.4848703e-09, 1.6018772e-09,
         1.3671555e-06, 5.7539902e-12, 9.6494257e-03, 1.3053580e-06,
         8.4899746e-08, 1.0012474e-05, 9.9135459e-06, 8.0573704e-14,
         1.2647015e-10, 1.5042176e-07, 8.7801961e-09, 1.3244300e-05,
         1.2191177e-06, 1.1181098e-08, 2.6688406e-12, 5.0014759e-10,
         3.3040468e-10, 7.7814423e-07, 5.0529407e-09, 8.0977603e-12,
         5.4619547e-14, 2.4976288e-10, 2.0288585e-10, 7.3272407e-15,
         7.5115366e-08, 2.7199059e-16, 3.5329208e-13, 4.2616280e-10,
         1.9717996e-07, 3.1666303e-09, 6.1058316e-05, 1.9999775e-06,
         9.8939347e-01, 1.7074611e-09, 5.4037452e-12, 5.1665083e-12,
         1.2732916e-10, 5.8817169e-09, 8.119

In [72]:
attention_distribution_group_p_gen = [(1 - p_gen) * attention_distribution for p_gen, attention_distribution in zip(p_gen_group, attention_distribution_group)]
attention_distribution_group_p_gen

[<tf.Tensor: id=22911, shape=(3, 80), dtype=float32, numpy=
 array([[4.79814560e-07, 2.93559932e-09, 1.82724902e-09, 1.12545466e-12,
         6.62421058e-17, 6.33429753e-10, 1.81853921e-09, 7.17684662e-11,
         3.37491363e-12, 1.77216470e-14, 7.01403416e-18, 1.27084052e-10,
         4.36369552e-10, 1.11055499e-13, 5.40098744e-10, 1.92908994e-10,
         1.64642202e-07, 6.92934805e-13, 1.16204983e-03, 1.57200120e-07,
         1.02242073e-08, 1.20577056e-06, 1.19385697e-06, 9.70323590e-15,
         1.52304003e-11, 1.81148163e-08, 1.05737119e-09, 1.59496915e-06,
         1.46814486e-07, 1.34650424e-09, 3.21400025e-13, 6.02311881e-11,
         3.97895883e-11, 9.37094455e-08, 6.08509632e-10, 9.75187596e-13,
         6.57765934e-15, 3.00781518e-11, 2.44329001e-11, 8.82396414e-16,
         9.04590536e-09, 3.27549659e-17, 4.25458449e-14, 5.13214367e-11,
         2.37457574e-08, 3.81347259e-10, 7.35305957e-06, 2.40850966e-07,
         1.19149521e-01, 2.05624129e-10, 6.50755925e-13, 6.22186

In [73]:
p_gen_group

[<tf.Tensor: id=4939, shape=(3, 1), dtype=float32, numpy=
 array([[0.87957317],
        [0.7754687 ],
        [0.8110325 ]], dtype=float32)>,
 <tf.Tensor: id=5840, shape=(3, 1), dtype=float32, numpy=
 array([[0.33255798],
        [0.6392306 ],
        [0.23890685]], dtype=float32)>,
 <tf.Tensor: id=6699, shape=(3, 1), dtype=float32, numpy=
 array([[0.2316914],
        [0.486628 ],
        [0.6512805]], dtype=float32)>,
 <tf.Tensor: id=7558, shape=(3, 1), dtype=float32, numpy=
 array([[0.5674238 ],
        [0.38187414],
        [0.6187975 ]], dtype=float32)>,
 <tf.Tensor: id=8417, shape=(3, 1), dtype=float32, numpy=
 array([[0.47214603],
        [0.4381661 ],
        [0.52042013]], dtype=float32)>,
 <tf.Tensor: id=9276, shape=(3, 1), dtype=float32, numpy=
 array([[0.45183408],
        [0.33085093],
        [0.3852602 ]], dtype=float32)>,
 <tf.Tensor: id=10135, shape=(3, 1), dtype=float32, numpy=
 array([[0.26367372],
        [0.5515943 ],
        [0.41685528]], dtype=float32)>,
 <tf.Ten

In [74]:
vocab_distribution_group

[<tf.Tensor: id=22720, shape=(3, 30000), dtype=float32, numpy=
 array([[3.6083133e-05, 3.3223176e-05, 3.2207736e-05, ..., 3.1815132e-05,
         3.5000299e-05, 3.2811677e-05],
        [3.5131383e-05, 3.1774151e-05, 3.1519558e-05, ..., 3.2256070e-05,
         3.3591517e-05, 3.2914653e-05],
        [3.5282690e-05, 3.4180499e-05, 2.9937295e-05, ..., 3.3263401e-05,
         3.4289798e-05, 3.0833853e-05]], dtype=float32)>,
 <tf.Tensor: id=22721, shape=(3, 30000), dtype=float32, numpy=
 array([[3.2740667e-05, 3.4407105e-05, 3.1988369e-05, ..., 3.2331664e-05,
         3.2501637e-05, 3.5359582e-05],
        [3.3276669e-05, 3.3870558e-05, 3.2061878e-05, ..., 3.2346634e-05,
         3.0804022e-05, 3.3469983e-05],
        [3.4126602e-05, 3.3000870e-05, 3.3177184e-05, ..., 3.1830044e-05,
         3.3714336e-05, 3.4968536e-05]], dtype=float32)>,
 <tf.Tensor: id=22722, shape=(3, 30000), dtype=float32, numpy=
 array([[3.0480602e-05, 3.1117412e-05, 3.2466229e-05, ..., 3.3706750e-05,
         3.261484

In [75]:
vocab_distribution_group_p_gen = [p_gen * vocab_distribution for p_gen, vocab_distribution in zip(p_gen_group, vocab_distribution_group)]
vocab_distribution_group_p_gen

[<tf.Tensor: id=23015, shape=(3, 30000), dtype=float32, numpy=
 array([[3.1737756e-05, 2.9222214e-05, 2.8329061e-05, ..., 2.7983737e-05,
         3.0785322e-05, 2.8860270e-05],
        [2.7243288e-05, 2.4639859e-05, 2.4442430e-05, ..., 2.5013573e-05,
         2.6049171e-05, 2.5524283e-05],
        [2.8615408e-05, 2.7721495e-05, 2.4280118e-05, ..., 2.6977697e-05,
         2.7810140e-05, 2.5007255e-05]], dtype=float32)>,
 <tf.Tensor: id=23016, shape=(3, 30000), dtype=float32, numpy=
 array([[1.08881695e-05, 1.14423574e-05, 1.06379875e-05, ...,
         1.07521528e-05, 1.08086788e-05, 1.17591107e-05],
        [2.12714658e-05, 2.16510980e-05, 2.04949338e-05, ...,
         2.06769582e-05, 1.96908732e-05, 2.13950370e-05],
        [8.15307885e-06, 7.88413399e-06, 7.92625633e-06, ...,
         7.60441571e-06, 8.05458512e-06, 8.35422270e-06]], dtype=float32)>,
 <tf.Tensor: id=23017, shape=(3, 30000), dtype=float32, numpy=
 array([[7.0620936e-06, 7.2096368e-06, 7.5221460e-06, ..., 7.8095645e-06,

In [76]:
attention_length_group = [tf.shape(attention_distribution)[1] for attention_distribution in attention_distribution_group]
attention_length_group

[<tf.Tensor: id=23061, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23066, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23071, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23076, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23081, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23086, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23091, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23096, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23101, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23106, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23111, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23116, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23121, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23126, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23131, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23136, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23141, shape=(), dtype=int32, numpy=80>,
 <tf.Tensor: id=23146, shape=()

In [77]:
attention_distribution = attention_distribution_group_p_gen[0]
attention_distribution

<tf.Tensor: id=22911, shape=(3, 80), dtype=float32, numpy=
array([[4.79814560e-07, 2.93559932e-09, 1.82724902e-09, 1.12545466e-12,
        6.62421058e-17, 6.33429753e-10, 1.81853921e-09, 7.17684662e-11,
        3.37491363e-12, 1.77216470e-14, 7.01403416e-18, 1.27084052e-10,
        4.36369552e-10, 1.11055499e-13, 5.40098744e-10, 1.92908994e-10,
        1.64642202e-07, 6.92934805e-13, 1.16204983e-03, 1.57200120e-07,
        1.02242073e-08, 1.20577056e-06, 1.19385697e-06, 9.70323590e-15,
        1.52304003e-11, 1.81148163e-08, 1.05737119e-09, 1.59496915e-06,
        1.46814486e-07, 1.34650424e-09, 3.21400025e-13, 6.02311881e-11,
        3.97895883e-11, 9.37094455e-08, 6.08509632e-10, 9.75187596e-13,
        6.57765934e-15, 3.00781518e-11, 2.44329001e-11, 8.82396414e-16,
        9.04590536e-09, 3.27549659e-17, 4.25458449e-14, 5.13214367e-11,
        2.37457574e-08, 3.81347259e-10, 7.35305957e-06, 2.40850966e-07,
        1.19149521e-01, 2.05624129e-10, 6.50755925e-13, 6.22186222e-13,
     

In [78]:
tf.reduce_sum(attention_distribution, -1)

<tf.Tensor: id=23184, shape=(3,), dtype=float32, numpy=array([0.12042683, 0.22453132, 0.18896751], dtype=float32)>

In [79]:
vocab_distribution = vocab_distribution_group_p_gen[0]
vocab_distribution

<tf.Tensor: id=23015, shape=(3, 30000), dtype=float32, numpy=
array([[3.1737756e-05, 2.9222214e-05, 2.8329061e-05, ..., 2.7983737e-05,
        3.0785322e-05, 2.8860270e-05],
       [2.7243288e-05, 2.4639859e-05, 2.4442430e-05, ..., 2.5013573e-05,
        2.6049171e-05, 2.5524283e-05],
       [2.8615408e-05, 2.7721495e-05, 2.4280118e-05, ..., 2.6977697e-05,
        2.7810140e-05, 2.5007255e-05]], dtype=float32)>

In [80]:
tf.reduce_sum(vocab_distribution, -1)

<tf.Tensor: id=23187, shape=(3,), dtype=float32, numpy=array([0.8795731 , 0.77546906, 0.8110326 ], dtype=float32)>

In [81]:
attention_length = attention_length_group[0]

In [82]:
batch_indices = tf.tile(tf.expand_dims(tf.range(0, batch_size), axis=1), [1, attention_length])
batch_indices

<tf.Tensor: id=23194, shape=(3, 80), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]], dtype=int32)>

In [83]:
indices = tf.stack((batch_indices, encoder_inputs_extend), axis=2)
indices

<tf.Tensor: id=23196, shape=(3, 80, 2), dtype=int32, numpy=
array([[[    0,   171],
        [    0,     3],
        [    0, 12606],
        [    0,  4045],
        [    0,  8484],
        [    0, 30000],
        [    0,   971],
        [    0, 30001],
        [    0,  7331],
        [    0,  1885],
        [    0, 10703],
        [    0,     3],
        [    0,   599],
        [    0, 16866],
        [    0,     5],
        [    0,   144],
        [    0,   108],
        [    0,     3],
        [    0, 30001],
        [    0,    87],
        [    0,    79],
        [    0,     4],
        [    0,   484],
        [    0, 14891],
        [    0,     5],
        [    0,    78],
        [    0, 19803],
        [    0, 12730],
        [    0,   541],
        [    0,  1055],
        [    0,  1338],
        [    0,     3],
        [    0,   492],
        [    0, 12334],
        [    0,   484],
        [    0,     5],
        [    0,  4072],
        [    0,    16],
        [    0,     8],
    

In [84]:
shape_extend = [batch_size, encoder_vocab_size + oovs_max_size]
shape_extend

[3, 30006]

In [85]:
attention_distribution_nd = tf.scatter_nd(indices, attention_distribution, shape_extend)
attention_distribution_nd

<tf.Tensor: id=23199, shape=(3, 30006), dtype=float32, numpy=
array([[0.0000000e+00, 9.9157858e-14, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 6.4289868e-15, 0.0000000e+00, ..., 1.3075924e-01,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 5.5248879e-17, 0.0000000e+00, ..., 7.6749177e-07,
        9.4192300e-11, 2.7425485e-11]], dtype=float32)>

In [86]:
attention_distribution_nd[0][30000]

<tf.Tensor: id=23208, shape=(), dtype=float32, numpy=6.3342975e-10>

In [87]:
vocab_distribution_nd = tf.concat([vocab_distribution, tf.zeros(shape=[batch_size, oovs_max_size])], axis=-1)

In [88]:
vocab_distribution_nd

<tf.Tensor: id=23212, shape=(3, 30006), dtype=float32, numpy=
array([[3.1737756e-05, 2.9222214e-05, 2.8329061e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.7243288e-05, 2.4639859e-05, 2.4442430e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.8615408e-05, 2.7721495e-05, 2.4280118e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)>

In [89]:
vocab_distribution_nd[0][30000]

<tf.Tensor: id=23221, shape=(), dtype=float32, numpy=0.0>

In [90]:
vocab_distribution_nd[0][29999]

<tf.Tensor: id=23230, shape=(), dtype=float32, numpy=2.886027e-05>

In [91]:
final_distribution_nd = attention_distribution_nd + vocab_distribution_nd

In [92]:
final_distribution_nd

<tf.Tensor: id=23232, shape=(3, 30006), dtype=float32, numpy=
array([[3.1737756e-05, 2.9222214e-05, 2.8329061e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.7243288e-05, 2.4639859e-05, 2.4442430e-05, ..., 1.3075924e-01,
        0.0000000e+00, 0.0000000e+00],
       [2.8615408e-05, 2.7721495e-05, 2.4280118e-05, ..., 7.6749177e-07,
        9.4192300e-11, 2.7425485e-11]], dtype=float32)>

In [93]:
tf.argmax(final_distribution_nd, -1)

<tf.Tensor: id=23234, shape=(3,), dtype=int64, numpy=array([ 7341, 30003,   281])>

In [94]:
tf.reduce_sum(final_distribution_nd, -1)

<tf.Tensor: id=23236, shape=(3,), dtype=float32, numpy=array([0.99999964, 1.0000004 , 0.99999994], dtype=float32)>

In [95]:
decoder_masks = tf.sequence_mask(lengths=decoder_inputs_train_length,
                                                      maxlen=decoder_max_time_steps + 1,
                                                      dtype=tf.float32,
                                                      name='masks')

In [96]:
decoder_masks

<tf.Tensor: id=23243, shape=(3, 21), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0.]], dtype=float32)>

In [97]:
final_distribution_nd

<tf.Tensor: id=23232, shape=(3, 30006), dtype=float32, numpy=
array([[3.1737756e-05, 2.9222214e-05, 2.8329061e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.7243288e-05, 2.4639859e-05, 2.4442430e-05, ..., 1.3075924e-01,
        0.0000000e+00, 0.0000000e+00],
       [2.8615408e-05, 2.7721495e-05, 2.4280118e-05, ..., 7.6749177e-07,
        9.4192300e-11, 2.7425485e-11]], dtype=float32)>

In [98]:
tf.reduce_sum(final_distribution_nd, -1)

<tf.Tensor: id=23246, shape=(3,), dtype=float32, numpy=array([0.99999964, 1.0000004 , 0.99999994], dtype=float32)>

In [111]:
final_dists = [final_distribution_nd for _ in range(decoder_max_time_steps + 1)]

In [112]:
final_dists

[<tf.Tensor: id=23232, shape=(3, 30006), dtype=float32, numpy=
 array([[3.1737756e-05, 2.9222214e-05, 2.8329061e-05, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [2.7243288e-05, 2.4639859e-05, 2.4442430e-05, ..., 1.3075924e-01,
         0.0000000e+00, 0.0000000e+00],
        [2.8615408e-05, 2.7721495e-05, 2.4280118e-05, ..., 7.6749177e-07,
         9.4192300e-11, 2.7425485e-11]], dtype=float32)>,
 <tf.Tensor: id=23232, shape=(3, 30006), dtype=float32, numpy=
 array([[3.1737756e-05, 2.9222214e-05, 2.8329061e-05, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [2.7243288e-05, 2.4639859e-05, 2.4442430e-05, ..., 1.3075924e-01,
         0.0000000e+00, 0.0000000e+00],
        [2.8615408e-05, 2.7721495e-05, 2.4280118e-05, ..., 7.6749177e-07,
         9.4192300e-11, 2.7425485e-11]], dtype=float32)>,
 <tf.Tensor: id=23232, shape=(3, 30006), dtype=float32, numpy=
 array([[3.1737756e-05, 2.9222214e-05, 2.8329061e-05, ..., 0.0000000e+00,
         0.000000

In [113]:
for dec_step in range(decoder_max_time_steps + 1) :
    print(dec_step)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [102]:
decoder_targets_train

<tf.Tensor: id=27, shape=(3, 21), dtype=int32, numpy=
array([[30000,    54,     2,  2350,     2,   541,  9438,   541,  1338,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1],
       [   44, 12257,   918,   610,     2,  1665,  5341,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1],
       [  487,   150, 26962,  2049, 30002,     2, 30003,    21,    42,
           17,   400,    60,   196,    63, 13789,     1,     1,     1,
            1,     1,     1]], dtype=int32)>

In [114]:
batch_nums = tf.range(0, batch_size)
losses_group = []
for dec_step in range(decoder_max_time_steps + 1) :
    final_dist = final_dists[dec_step]
    targets = decoder_targets_train[:, dec_step]
    indices = tf.stack((batch_nums, targets), axis=1)
    print(dec_step)
    print(final_dist)
    print(targets)
    print('indices', indices)
    gold_probs = tf.gather_nd(final_dist, indices)
    print('Glod Probs', gold_probs)
    losses = -tf.log(gold_probs)
    print('Losses', losses)
    losses_group.append(losses)
    print('*' * 20)
losses_group

0
tf.Tensor(
[[3.1737756e-05 2.9222214e-05 2.8329061e-05 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.7243288e-05 2.4639859e-05 2.4442430e-05 ... 1.3075924e-01
  0.0000000e+00 0.0000000e+00]
 [2.8615408e-05 2.7721495e-05 2.4280118e-05 ... 7.6749177e-07
  9.4192300e-11 2.7425485e-11]], shape=(3, 30006), dtype=float32)
tf.Tensor([30000    44   487], shape=(3,), dtype=int32)
indices tf.Tensor(
[[    0 30000]
 [    1    44]
 [    2   487]], shape=(3, 2), dtype=int32)
Glod Probs tf.Tensor([6.3342975e-10 2.5122912e-05 2.6915865e-05], shape=(3,), dtype=float32)
Losses tf.Tensor([21.179873 10.59173  10.522795], shape=(3,), dtype=float32)
********************
1
tf.Tensor(
[[3.1737756e-05 2.9222214e-05 2.8329061e-05 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.7243288e-05 2.4639859e-05 2.4442430e-05 ... 1.3075924e-01
  0.0000000e+00 0.0000000e+00]
 [2.8615408e-05 2.7721495e-05 2.4280118e-05 ... 7.6749177e-07
  9.4192300e-11 2.7425485e-11]], shape=(3, 30006), dtype=float32)
tf.Ten

[<tf.Tensor: id=24337, shape=(3,), dtype=float32, numpy=array([21.179873, 10.59173 , 10.522795], dtype=float32)>,
 <tf.Tensor: id=24350, shape=(3,), dtype=float32, numpy=array([10.394659, 10.567746, 10.49488 ], dtype=float32)>,
 <tf.Tensor: id=24363, shape=(3,), dtype=float32, numpy=array([10.471622, 10.642588, 10.562646], dtype=float32)>,
 <tf.Tensor: id=24376, shape=(3,), dtype=float32, numpy=array([10.50076 , 10.533116, 10.574855], dtype=float32)>,
 <tf.Tensor: id=24389, shape=(3,), dtype=float32, numpy=array([10.471622, 10.61919 , 19.226612], dtype=float32)>,
 <tf.Tensor: id=24402, shape=(3,), dtype=float32, numpy=array([10.387898, 10.506787, 10.625853], dtype=float32)>,
 <tf.Tensor: id=24415, shape=(3,), dtype=float32, numpy=array([10.4541445, 10.516265 , 14.080138 ], dtype=float32)>,
 <tf.Tensor: id=24428, shape=(3,), dtype=float32, numpy=array([10.387898 , 10.611145 , 10.4991865], dtype=float32)>,
 <tf.Tensor: id=24441, shape=(3,), dtype=float32, numpy=array([10.411907, 10.61114

In [115]:
len(losses_group)

21

In [109]:
decoder_masks

<tf.Tensor: id=23243, shape=(3, 21), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0.]], dtype=float32)>

In [116]:
dec_lens = tf.reduce_sum(decoder_masks, axis=1)

In [117]:
dec_lens

<tf.Tensor: id=24620, shape=(3,), dtype=float32, numpy=array([10.,  8., 16.], dtype=float32)>

In [118]:
values_per_step = [v * decoder_masks[:, dec_step] for dec_step, v in enumerate(losses_group)]

In [119]:
values_per_step

[<tf.Tensor: id=24626, shape=(3,), dtype=float32, numpy=array([21.179873, 10.59173 , 10.522795], dtype=float32)>,
 <tf.Tensor: id=24631, shape=(3,), dtype=float32, numpy=array([10.394659, 10.567746, 10.49488 ], dtype=float32)>,
 <tf.Tensor: id=24636, shape=(3,), dtype=float32, numpy=array([10.471622, 10.642588, 10.562646], dtype=float32)>,
 <tf.Tensor: id=24641, shape=(3,), dtype=float32, numpy=array([10.50076 , 10.533116, 10.574855], dtype=float32)>,
 <tf.Tensor: id=24646, shape=(3,), dtype=float32, numpy=array([10.471622, 10.61919 , 19.226612], dtype=float32)>,
 <tf.Tensor: id=24651, shape=(3,), dtype=float32, numpy=array([10.387898, 10.506787, 10.625853], dtype=float32)>,
 <tf.Tensor: id=24656, shape=(3,), dtype=float32, numpy=array([10.4541445, 10.516265 , 14.080138 ], dtype=float32)>,
 <tf.Tensor: id=24661, shape=(3,), dtype=float32, numpy=array([10.387898 , 10.611145 , 10.4991865], dtype=float32)>,
 <tf.Tensor: id=24666, shape=(3,), dtype=float32, numpy=array([10.411907,  0.     

In [120]:
sum(values_per_step)

<tf.Tensor: id=24768, shape=(3,), dtype=float32, numpy=array([115.10097,  84.58857, 179.54161], dtype=float32)>

In [123]:
total = sum(values_per_step) / dec_lens

In [124]:
total

<tf.Tensor: id=24835, shape=(3,), dtype=float32, numpy=array([11.510097, 10.573571, 11.221351], dtype=float32)>

In [125]:
tf.reduce_mean(total)

<tf.Tensor: id=24838, shape=(), dtype=float32, numpy=11.101673>

In [126]:
vocab_distribution_group

[<tf.Tensor: id=22720, shape=(3, 30000), dtype=float32, numpy=
 array([[3.6083133e-05, 3.3223176e-05, 3.2207736e-05, ..., 3.1815132e-05,
         3.5000299e-05, 3.2811677e-05],
        [3.5131383e-05, 3.1774151e-05, 3.1519558e-05, ..., 3.2256070e-05,
         3.3591517e-05, 3.2914653e-05],
        [3.5282690e-05, 3.4180499e-05, 2.9937295e-05, ..., 3.3263401e-05,
         3.4289798e-05, 3.0833853e-05]], dtype=float32)>,
 <tf.Tensor: id=22721, shape=(3, 30000), dtype=float32, numpy=
 array([[3.2740667e-05, 3.4407105e-05, 3.1988369e-05, ..., 3.2331664e-05,
         3.2501637e-05, 3.5359582e-05],
        [3.3276669e-05, 3.3870558e-05, 3.2061878e-05, ..., 3.2346634e-05,
         3.0804022e-05, 3.3469983e-05],
        [3.4126602e-05, 3.3000870e-05, 3.3177184e-05, ..., 3.1830044e-05,
         3.3714336e-05, 3.4968536e-05]], dtype=float32)>,
 <tf.Tensor: id=22722, shape=(3, 30000), dtype=float32, numpy=
 array([[3.0480602e-05, 3.1117412e-05, 3.2466229e-05, ..., 3.3706750e-05,
         3.261484

In [127]:
len(vocab_distribution_group)

21

In [128]:
decoder_masks

<tf.Tensor: id=23243, shape=(3, 21), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0.]], dtype=float32)>

In [135]:
coverage = tf.zeros_like(vocab_distribution_group[0])
covlosses = []  # Coverage loss per decoder timestep. Will be list length max_dec_steps containing shape (batch_size).
for a in vocab_distribution_group:
    print('a', a)
    print('min', tf.minimum(a, coverage))
    covloss = tf.reduce_sum(tf.minimum(a, coverage), [1])  # calculate the coverage loss for this step
    print('covloss', covloss)
    covlosses.append(covloss)
    coverage += a  # update the coverage vector
    print('coverage', coverage)
dec_lens = tf.reduce_sum(decoder_masks, axis=1)
values_per_step = [v * decoder_masks[:, dec_step] for dec_step, v in enumerate(covlosses)]
values_per_ex = sum(values_per_step) / dec_lens  # shape (batch_size); normalized value for each batch member
loss = tf.reduce_mean(values_per_ex)
print(loss)

a tf.Tensor(
[[3.6083133e-05 3.3223176e-05 3.2207736e-05 ... 3.1815132e-05
  3.5000299e-05 3.2811677e-05]
 [3.5131383e-05 3.1774151e-05 3.1519558e-05 ... 3.2256070e-05
  3.3591517e-05 3.2914653e-05]
 [3.5282690e-05 3.4180499e-05 2.9937295e-05 ... 3.3263401e-05
  3.4289798e-05 3.0833853e-05]], shape=(3, 30000), dtype=float32)
min tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(3, 30000), dtype=float32)
covloss tf.Tensor([0. 0. 0.], shape=(3,), dtype=float32)
coverage tf.Tensor(
[[3.6083133e-05 3.3223176e-05 3.2207736e-05 ... 3.1815132e-05
  3.5000299e-05 3.2811677e-05]
 [3.5131383e-05 3.1774151e-05 3.1519558e-05 ... 3.2256070e-05
  3.3591517e-05 3.2914653e-05]
 [3.5282690e-05 3.4180499e-05 2.9937295e-05 ... 3.3263401e-05
  3.4289798e-05 3.0833853e-05]], shape=(3, 30000), dtype=float32)
a tf.Tensor(
[[3.2740667e-05 3.4407105e-05 3.1988369e-05 ... 3.2331664e-05
  3.2501637e-05 3.5359582e-05]
 [3.3276669e-05 3.3870558e-05 3.2061878e-05 ... 3.2

In [ ]:
dec_lens = tf.reduce_sum(decoder_masks, axis=1)